In [1]:
import requests
import json

admin = {
    'faber': 'http://0.0.0.0:8021',
    'alice': 'http://0.0.0.0:8031'
}

In [4]:
import requests

dtype = 'ssn'

schema_body = {
    "schema_name": dtype,
    "schema_version": '1.1.1',
    "attributes": [dtype, 'issuer', 'holder'],
}    
schema_response = requests.post(
    admin['faber'] + "/schemas", 
    json=schema_body
).json()

schema_id = schema_response["schema_id"]

credential_definition_body = {
    "schema_id": schema_id,
    "support_revocation": False,
    "revocation_registry_size": 4,
    "tag": 'default',
}
credential_definition_response = requests.post(
    admin['faber'] + "/credential-definitions", 
    json=credential_definition_body
).json()

credential_definition_id = credential_definition_response[
    "credential_definition_id"
]    
print(
    schema_id, credential_definition_id
)

HcnMNFeUxjX1AzpDuP7ZzD:2:ssn:1.1.1 HcnMNFeUxjX1AzpDuP7ZzD:3:CL:52017:default


In [3]:
credential_definition_response

{'revocation_registry_size': ['Value 0 must be an integer between 4 and 32768 inclusively']}

In [2]:
connections = requests.get(
    admin['faber'] + '/connections'
).json()
connection_id = connections['results'][0]['connection_id']

issuer_did = requests.get(
    admin['faber'] + '/wallet/did/public'
).json()['result']['did']

schema_ids = requests.get(
    admin['faber'] + '/schemas/created'
).json()['schema_ids']

credential_definition_ids = requests.get(
    admin['faber'] + '/credential-definitions/created'
).json()['credential_definition_ids']

In [4]:
[
    x['connection_id']
    for x in connections['results']
]

['b04cbf80-08a3-42fd-9aa1-6f3a3d95f23f',
 'e0e92856-50c5-4603-8ed5-1f9c6792a0e5',
 'f1c3d906-0235-480e-8383-c4e800feb5e9',
 '25153111-7e45-4c65-a51a-73a1a0738de9']

In [29]:
cred_def_id = credential_definition_ids[0]
cred_def_id

'FtT6DqATds6ws9FNqREy6H:3:CL:52023:default'

In [17]:
credential_definition_ids

['FtT6DqATds6ws9FNqREy6H:3:CL:52023:default',
 'FtT6DqATds6ws9FNqREy6H:3:CL:52021:Faber.Agent.degree_schema']

In [20]:
connection_id

'2e2efaf8-93c5-4a7d-afd4-8c2f597918f7'

In [32]:
json.dumps(offer_request)

'{"connection_id": "2e2efaf8-93c5-4a7d-afd4-8c2f597918f7", "auto_remove": false, "credential_preview": {"@type": "https://didcomm.org/issue-credential/2.0/credential-preview", "attributes": [{"name": "holder", "value": "Alice Smith"}, {"name": "issuer", "value": "FtT6DqATds6ws9FNqREy6H"}, {"name": "ssn", "value": "123456789"}]}, "filter": {"indy": {"cred_def_id": "FtT6DqATds6ws9FNqREy6H:3:CL:52023:default"}}, "trace": false}'

In [31]:
offer_request = {
    "connection_id": connection_id,
    "auto_remove": False,
    "credential_preview": {
        "@type": "https://didcomm.org/issue-credential/2.0/credential-preview",
        "attributes": [
            {
                "name": "holder",
                "value": "Alice Smith"
            },
            {
                "name": "issuer",
                "value": issuer_did
            },
            {
                "name": "ssn",
                "value": "123456789"
            }
        ]
    },
    "filter": {
        "indy": {
            "cred_def_id": cred_def_id
        }
    },
    "trace": False
}

requests.post(
    admin['faber'] + "/issue-credential-2.0/send-offer", 
    json=offer_request
).json()

{'cred_offer': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/2.0/offer-credential',
  '@id': 'a8f65549-7b0f-438b-9d6d-80dafe2e4c6b',
  '~thread': {},
  'credential_preview': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/2.0/credential-preview',
   'attributes': [{'name': 'holder', 'value': 'Alice Smith'},
    {'name': 'issuer', 'value': 'FtT6DqATds6ws9FNqREy6H'},
    {'name': 'ssn', 'value': '123456789'}]},
  'formats': [{'attach_id': '0', 'format': 'hlindy-zkp-v1.0'}],
  'offers~attach': [{'@id': '0',
    'mime-type': 'application/json',
    'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiRnRUNkRxQVRkczZ3czlGTnFSRXk2SDoyOnNzbjoxLjEuMSIsICJjcmVkX2RlZl9pZCI6ICJGdFQ2RHFBVGRzNndzOUZOcVJFeTZIOjM6Q0w6NTIwMjM6ZGVmYXVsdCIsICJrZXlfY29ycmVjdG5lc3NfcHJvb2YiOiB7ImMiOiAiNDM4NTUxMzMxNTQyODM5Mjc5NzE0OTkwNDM3Njc1MDg0NDY4MjY2MTAxMjQ3NTUxNTk2OTMyNDQxMzc5ODIzNDE3OTE2NDIwMjk0NjIiLCAieHpfY2FwIjogIjY0NTYwNjg5Mzk0NDQzODg5MTIyNjgyMTQwMDU5NjI4NDc5MTczMjM3MDI0NjY1NDc3ODkwODk4MTcwODA

In [36]:
offer_request =   {
    "connection_id": "d33f1463-e0aa-4280-b097-05712cb757ad",
    "proof_request": {
      "name": "Proof of Education",
      "version": "1.0",
      "requested_attributes": {
        "0_name_uuid": {
          "name": "name",
          "restrictions": [
            {
              "schema_name": "degree schema"
            }
          ]
        },
        "0_date_uuid": {
          "name": "date",
          "restrictions": [
            {
              "schema_name": "degree schema"
            }
          ]
        },
        "0_degree_uuid": {
          "name": "degree",
          "restrictions": [
            {
              "schema_name": "degree schema"
            }
          ]
        }
      ]
    },
    "filter": {
      "indy": {
        "cred_def_id": "FtT6DqATds6ws9FNqREy6H:3:CL:52021:Faber.Agent.degree_schema"
      }
    },
    "trace": False
  }

requests.post(
    admin['faber'] + "/issue-credential-2.0/send-offer", 
    json=offer_request
).json()

SyntaxError: invalid syntax (<ipython-input-36-197ec313bc57>, line 31)

In [35]:
proof_request = {
    "connection_id": connection_id,
    "proof_request": {
        "name": "Proof of Education",
        "version": "1.0",
        "requested_attributes": {
            "0_name_uuid": {
                "name": "name"
            }
        },
        "requested_predicates": {}
    },
    "trace": False
}
  
requests.post(
    admin['faber'] + "/present-proof/send-request", 
    json=proof_request
).json()

{'presentation_request': {'name': 'Proof of Education',
  'version': '1.0',
  'requested_attributes': {'0_name_uuid': {'name': 'name'}},
  'requested_predicates': {},
  'nonce': '961572405810211622919343'},
 'trace': False,
 'auto_present': False,
 'thread_id': '8b0708c4-4d3e-4613-a1fc-4a4dee3c19c1',
 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',
  '@id': '8b0708c4-4d3e-4613-a1fc-4a4dee3c19c1',
  'request_presentations~attach': [{'@id': 'libindy-request-presentation-0',
    'mime-type': 'application/json',
    'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIEVkdWNhdGlvbiIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9uYW1lX3V1aWQiOiB7Im5hbWUiOiAibmFtZSJ9fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25jZSI6ICI5NjE1NzI0MDU4MTAyMTE2MjI5MTkzNDMifQ=='}}]},
 'state': 'request_sent',
 'role': 'verifier',
 'presentation_exchange_id': '41766e7a-e1ff-434b-9cca-48f1349827c4',
 'created_at': '2021-03-03 01:00:59.38

In [5]:
creds = {'results': [{'attrs': {'age': '24',
                        'date': '2018-05-28',
                        'degree': 'Maths',
                        'name': 'Alice Smith',
                        'timestamp': '1614799320'},
              'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52151:Faber.Agent.degree_schema',
              'cred_rev_id': None,
              'referent': '516ded46-3bf4-4171-ae6f-e43d7063e7f8',
              'rev_reg_id': None,
              'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:degree schema:36.100.6'},
             {'attrs': {'holder': 'George',
                        'issuer': 'MadyBKDn2giCmKRF8xGuFp',
                        'ssn': '1',
                        'timestamp': '1614800846'},
              'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
              'cred_rev_id': None,
              'referent': '810af234-ce9c-48d6-b4c4-6afd0de305a6',
              'rev_reg_id': None,
              'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'},
             {'attrs': {'holder': 'George',
                        'issuer': 'MadyBKDn2giCmKRF8xGuFp',
                        'ssn': '1',
                        'timestamp': '1614799973'},
              'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
              'cred_rev_id': None,
              'referent': 'd7e562da-68e7-4996-897b-06a7cc992c21',
              'rev_reg_id': None,
              'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'},
             {'attrs': {'holder': 'George',
                        'issuer': 'MadyBKDn2giCmKRF8xGuFp',
                        'ssn': '1',
                        'timestamp': '1614800134'},
              'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
              'cred_rev_id': None,
              'referent': '1415ad74-c0db-487c-ad78-e62baaf6d04c',
              'rev_reg_id': None,
              'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'}]}

In [8]:
sorted(
    creds['results'],
    key = lambda x: int(x['attrs']['timestamp']),
    reverse=True
)

[{'attrs': {'holder': 'George',
   'issuer': 'MadyBKDn2giCmKRF8xGuFp',
   'ssn': '1',
   'timestamp': '1614800846'},
  'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
  'cred_rev_id': None,
  'referent': '810af234-ce9c-48d6-b4c4-6afd0de305a6',
  'rev_reg_id': None,
  'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'},
 {'attrs': {'holder': 'George',
   'issuer': 'MadyBKDn2giCmKRF8xGuFp',
   'ssn': '1',
   'timestamp': '1614800134'},
  'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
  'cred_rev_id': None,
  'referent': '1415ad74-c0db-487c-ad78-e62baaf6d04c',
  'rev_reg_id': None,
  'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'},
 {'attrs': {'holder': 'George',
   'issuer': 'MadyBKDn2giCmKRF8xGuFp',
   'ssn': '1',
   'timestamp': '1614799973'},
  'cred_def_id': 'MadyBKDn2giCmKRF8xGuFp:3:CL:52153:ssn',
  'cred_rev_id': None,
  'referent': 'd7e562da-68e7-4996-897b-06a7cc992c21',
  'rev_reg_id': None,
  'schema_id': 'MadyBKDn2giCmKRF8xGuFp:2:ssn:1.1.1'},
 {'attrs': {'age